In [ ]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM

from keras.layers import Dropout

In [ ]:
data = pd.read_csv('data/household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

def get_acf(data,lags): 
    frame = []
    for i in range(lags+1):
        frame.append(data.apply(lambda col: col.autocorr(i), axis=0))
    return pd.DataFrame(frame).plot.line()
get_acf(data,10)

In [ ]:
#fill in missing values using linear interpolation with limit to 10 using autocorelation plot
clean_data = data.interpolate(method = 'linear', axis = 0, limit = 10)

i = 1
# plot each column
plt.figure(figsize=(20, 15))
for counter in range(1,len(clean_data.columns)):
    plt.subplot(len(clean_data.columns), 1, i)
    plt.plot(clean_data.resample('D').mean().values[:, counter], color = 'blue')
    plt.title(clean_data.columns[counter], y=0.8, loc='right')
    i = i+1
plt.show()

In [ ]:
import seaborn as sns # used for plot interactive graph. 

#check corelation matrices for minute, hour and day
#minute
corr_min = clean_data.corr()
#hour
corr_hour = pd.DataFrame(clean_data.resample('H').mean().values).corr()
#day
corr_day = pd.DataFrame(clean_data.resample('D').mean().values).corr()
cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#Plot Minute
corr_min.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())



In [ ]:
corr_hour.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
corr_day.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
#Credit: Adopted from https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
resampled_data = clean_data.resample('H').mean() 

In [ ]:
reframed_data = series_to_supervised(resampled_data, 3, 1)
print(reframed_data.head())

In [ ]:
reframed_data.drop(reframed_data.columns[-6:], axis=1, inplace=True)

In [ ]:
print(reframed_data.columns)

In [ ]:
values = reframed_data.values
#The logic is to have 500 days worth of training data. this could also be a hyperparameter that can be tuned.
train_index = 800*24 
train = values[:train_index, :]
test = values[train_index:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 3, int(train_X.shape[1]/3)))
test_X = test_X.reshape((test_X.shape[0], 3, int(test_X.shape[1]/3)))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features].

In [ ]:
from keras.layers import Embedding
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=((3, train_X.shape[2]))))
#model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False, input_shape=((3, 200))))
#model.add(Dropout(0.1))
model.add(Dense(1))
model.summary()
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=10, batch_size=20, validation_data=(test_X, test_y), verbose=1, shuffle=False)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
yhat = model.predict(train_X, verbose=0)
rmse_train = np.sqrt(mean_squared_error(train_y, yhat))
yhat = model.predict(test_X, verbose=0)
rmse_test = np.sqrt(mean_squared_error(test_y, yhat))
print('Train RMSE: %.3f' % rmse_train)
print('Test RMSE: %.3f' % rmse_test)

In [ ]:
from keras.models import Model

model_for_xgb = model

intermediate_layer_model = Model(inputs=model_for_xgb.input,
                                 outputs=model_for_xgb.get_layer(index=1).output)

intermediate_train = intermediate_layer_model.predict(train_X)
intermediate_test = intermediate_layer_model.predict(test_X)


In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(intermediate_train, train_y)
param = {'max_depth': 4, 'objective':'reg:linear', 'eta': 0.1, 'tree_method': 'hist', 'nthread': 4,
         'max_bin': 256, "grow_policy": "lossguide", "max_leaves": 32}
bst = xgb.train(param, dtrain, 30)

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = bst.predict(xgb.DMatrix(intermediate_train, train_y))
rmse_train = np.sqrt(mean_squared_error(train_y, predictions))
print("Train RMSE: %.3f" % rmse_train)
predictions = bst.predict(xgb.DMatrix(intermediate_test, test_y))
rmse_test = np.sqrt(mean_squared_error(test_y, predictions))
print("Test RMSE: %.3f" % rmse_test)

In [ ]:
xgb.to_graphviz(bst, num_trees=1)